# Segementing and Clustering in Toronto

#### Import Packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
from lxml import html
import requests
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Retrieve Table from website

In [2]:
#Get page
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soupobj=BeautifulSoup(page,'lxml')
table=soupobj.find('table',{'class':'wikitable sortable'})

Postcodes=[]
Boroughs=[]
Neighbourhoods=[]

for tr in table.find_all('tr'):
    tds=tr.find_all('td')
    if len(tds)==3:
        Postcodes.append(tds[0].text)
        Boroughs.append(tds[1].text)
        Neighbourhoods.append(tds[2].text.strip())

        datatup=list(zip(Postcodes, Boroughs,Neighbourhoods))        



#### Create Pandas Data Frame

In [3]:
fulldf=pd.DataFrame(datatup,columns=['Postcodes','Borough','Neighborhood'])
df = fulldf[fulldf.Borough != 'Not assigned']
df=df.replace('Not assigned',np.nan)
df['Neighborhood']=df['Neighborhood'].fillna(df['Borough'])
Output=df.groupby(['Postcodes','Borough'])['Neighborhood'].apply(', '.join).reset_index()
Output

,Postcodes,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
Output.shape

(103, 3)

#### Retreiving Longitude and Latitude

In [5]:
# !conda install -c conda-forge geocoder 
# import geocoder
# lat_lng_coords= None
# while(lat_lng_coords is None):
#     g=geocoder.google('{},Toronto,Ontario'.format(Postcodes))
#     lat_lng_coords=g.latlng
    
# latitude=lay_lng_coords[0]
# longitude=lat_lng_coords[1]



In [6]:
loc_coord=pd.read_csv('https://cocl.us/Geospatial_data')
loc_hood_df=pd.merge(Output,loc_coord,how='left', left_on=['Postcodes'], right_on=['Postal Code'])
loc_hood_df=loc_hood_df.drop(['Postal Code'],axis=1)
loc_hood_df.head()

,Postcodes,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Limit to Boroughs containing word 'Toronto'

In [7]:
torontocenter_data = loc_hood_df[loc_hood_df['Borough'].str.contains("Downtown Toronto")].reset_index(drop=True)
torontocenter_data

,Postcodes,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


#### Create map of central Toronto 

In [8]:

#Lat Long for University of Toronto 43.661369 -79.396263

map_toronto = folium.Map(location=[43.661369 ,-79.396263],zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontocenter_data['Latitude'], torontocenter_data['Longitude'], torontocenter_data['Borough'], torontocenter_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=0.4,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Use Foursquare API to explore venues in Toronto city center

In [9]:
CLIENT_ID = 'NHALHKECTHEG32XKAQMZVYSIGPHDAXB2DFYZJRUHPDEXDSE1' # your Foursquare ID
CLIENT_SECRET = 'NVA1PYBHZ35VNBAHHFV4FCLPCPKWBTZHA2BYM5BYKLRG5KIC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
limit=300
radius=100

#### Function to scrape venues within 100 meters

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=torontocenter_data['Neighborhood'],
                                   latitudes=torontocenter_data['Latitude'],
                                   longitudes=torontocenter_data['Longitude']
                                  )


Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [12]:
toronto_venues.shape

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",11,11,11,11,11,11
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",1,1,1,1,1,1
"Cabbagetown, St. James Town",1,1,1,1,1,1
Central Bay Street,4,4,4,4,4,4
"Chinatown, Grange Park, Kensington Market",5,5,5,5,5,5
Christie,1,1,1,1,1,1
"Commerce Court, Victoria Hotel",16,16,16,16,16,16
"Design Exchange, Toronto Dominion Centre",7,7,7,7,7,7
"First Canadian Place, Underground city",12,12,12,12,12,12


#### Analyze neighborhoods

In [21]:
# one hot encoding
t_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

t_onehot.shape
t_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Nightclub,Park,Performing Arts Venue,Pub,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Group Neighborhoods

In [22]:
#Take mean of frequency of occurence 
t_grouped = t_onehot.groupby('Neighborhood').mean().reset_index()
t_grouped

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Nightclub,Park,Performing Arts Venue,Pub,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.0000,0.0000,0.000000,0.000000,0.090909,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.090909,0.000000,0.000000,0.0,0.0000,0.090909,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.181818,0.000000,0.090909,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.00,0.090909,0.090909,0.090909,0.000000,0.0,0.090909
1,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,1.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Cabbagetown, St. James Town",0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Central Bay Street,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.750000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.2500,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Chinatown, Grange Park, Kensington Market",0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.200000,0.2,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.000000
5,Christie,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Commerce Court, Victoria Hotel",0.0625,0.0625,0.062500,0.000000,0.000000,0.0625,0.000000,0.000000,0.0625,0.062500,0.0,0.062500,0.0,0.000000,0.000000,0.062500,0.0,0.0625,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.062500,0.000000,0.062500,0.0625,0.0625,0.0625,0.00,0.000000,0.062500,0.000000,0.000000,0.0,0.000000
7,"Design Exchange, Toronto Dominion Centre",0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0,0.428571,0.0,0.000000,0.000000,0.142857,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.285714,0.000000,0.0000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"First Canadian Place, Underground city",0.0000,0.0000,0.083333,0.083333,0.000000,0.0000,0.000000,0.083333,0.0000,0.166667,0.0,0.000000,0.0,0.000000,0.083333,0.083333,0.0,0.0000,0.000000,0.083333,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.08333

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = t_grouped['Neighborhood']

for ind in np.arange(t_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)



#### Cluster Neighborhoods using K Means

In [25]:
kclusters = 8

t_grouped_clustering = t_grouped.drop('Neighborhood', 1)

t_grouped_clustering
# # run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_grouped_clustering)
#kmeans.labels_[0:5] 
kmeans.labels_

array([1, 6, 0, 7, 1, 2, 1, 7, 1, 5, 3, 1, 4, 1, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# # neighborhoods_venues_sorted
t_merged = torontocenter_data

# # # # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), how='right',on='Neighborhood')

t_merged.head() # check the last columns!


,Postcodes,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Spa,Italian Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Coffee Shop,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Art Gallery,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Japanese Restaurant,Coffee Shop,Hostel,Italian Restaurant,Performing Arts Venue,Gastropub,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop


In [29]:
map_clusters = folium.Map(location=[43.661369 ,-79.396263], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['Neighborhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [30]:
t_merged.loc[t_merged['Cluster Labels'] == 0, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall


In [31]:
t_merged.loc[t_merged['Cluster Labels'] == 1, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,1,Spa,Italian Restaurant,Breakfast Spot,Park,Vegetarian / Vegan Restaurant,Coffee Shop,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega
4,Downtown Toronto,1,Art Gallery,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop
5,Downtown Toronto,1,Japanese Restaurant,Coffee Shop,Hostel,Italian Restaurant,Performing Arts Venue,Gastropub,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop
8,Downtown Toronto,1,Hotel,Vegetarian / Vegan Restaurant,Greek Restaurant,Tea Room,Sushi Restaurant,Steakhouse,Bar,Concert Hall,Food Court,Japanese Restaurant
11,Downtown Toronto,1,Gym,Pub,Art Gallery,Bakery,Bookstore,Burrito Place,Café,Coffee Shop,Deli / Bodega,Fast Food Restaurant
13,Downtown Toronto,1,Cocktail Bar,Café,Farmers Market,Liquor Store,Thrift / Vintage Store,Bar,Bookstore,Gastropub,Food Court,Fast Food Restaurant
15,Downtown Toronto,1,Pub,Steakhouse,Bar,Hotel,Breakfast Spot,Italian Restaurant,Concert Hall,Coffee Shop,Fast Food Restaurant,Farmers Market
16,Downtown Toronto,1,Café,Burger Joint,Gym / Fitness Center,Deli / Bodega,Cupcake Shop,Gluten-free Restaurant,Salad Place,Gastropub,Bank,Bakery


In [32]:
t_merged.loc[t_merged['Cluster Labels'] == 2, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Nightclub,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall


In [33]:
t_merged.loc[t_merged['Cluster Labels'] == 3, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,3,Gym,Greek Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall,College Gym


In [34]:
t_merged.loc[t_merged['Cluster Labels'] == 4, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Park,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall


In [35]:
t_merged.loc[t_merged['Cluster Labels'] == 5, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,5,College Gym,Vegetarian / Vegan Restaurant,Greek Restaurant,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall


In [36]:
t_merged.loc[t_merged['Cluster Labels'] == 6, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,6,Performing Arts Venue,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,Concert Hall


In [37]:
t_merged.loc[t_merged['Cluster Labels'] == 7, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,7,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Gastropub,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop
10,Downtown Toronto,7,Coffee Shop,Restaurant,Deli / Bodega,Gym,Bank,College Gym,Gastropub,Food Court,Fast Food Restaurant,Farmers Market
